In [1]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math, random
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import csv, torch
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm
from transformers import AdamW
import torch.nn.functional as F
from tqdm import trange


In [2]:
data = pd.read_csv('Data/IceKing.csv', sep="|")

cleanup = []
for index,row in data.iterrows():
 if type(row[1]) == float:
   cleanup.append(index)

for ind in cleanup:
 data = data.drop(labels = ind, axis = 0)

cleanup2 = []
for index,row in data.iterrows():
 if type(row[1]) == float:
   cleanup2.append(index)

for ind in cleanup2:
 data = data.drop(labels = ind, axis = 0)

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

model = GPT2LMHeadModel.from_pretrained('distilgpt2')

SPECIAL_TOKENS_DICT = {
    'pad_token': '<pad>',
    'additional_special_tokens': ['<context>', '<character>'],
}

tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [4]:
class CharacterDataset(Dataset):
  def __init__(self, data, tokenizer, seq_length=64):

    context_tkn = tokenizer.additional_special_tokens_ids[0]
    character_tkn = tokenizer.additional_special_tokens_ids[1]
    pad_tkn = tokenizer.pad_token_id
    eos_tkn = tokenizer.eos_token_id

    self.examples = []
    for index, row in data.iterrows():
      # Build the context and character segments:
      context = [context_tkn] + tokenizer.encode(row[0], max_length=seq_length // 2 - 1)
      character = [character_tkn] + tokenizer.encode(row[1], max_length=seq_length // 2 - 2) + [eos_tkn]

      # Concatenate the two parts together:
      tokens = context + character + [pad_tkn] * (seq_length - len(context) - len(character))

      # Annotate each token with its corresponding segment:
      segments = [context_tkn] * len(context) + [character_tkn] * (seq_length - len(context))

      # Ignore the context, padding, and <character> tokens by setting their labels to -1
      labels = [-100] * (len(context) + 1) + character[1:] + [-100] * (seq_length - len(context) - len(character))

      # Add the preprocessed example to the dataset
      self.examples.append((tokens, segments, labels))

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item])

In [5]:
character_dataset = CharacterDataset(data, tokenizer)

# Create data indices for training and validation splits:

indices = list(range(len(character_dataset)))

random.seed(42)
random.shuffle(indices)

split = math.floor(0.1 * len(character_dataset))
train_indices, val_indices = indices[split:], indices[:split]

# Build the PyTorch data loaders:
torch.manual_seed(0)
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(character_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(character_dataset, batch_size=64, sampler=val_sampler)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:

def fit(model, optimizer, train_dl, val_dl, epochs=1, device=torch.device('cuda')):
  for i in range(epochs):

    print('\n--- Starting epoch #{} ---'.format(i))

    model.train()

    # These 2 lists will keep track of the batch losses and batch sizes over one epoch:
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # Move the batch to the training device:

      inputs = xb.to(device)

      # Call the model with the token ids, segment ids, and the ground truth (labels)
      outputs = model(inputs[:, 0, :], token_type_ids=inputs[:, 1, :], labels=inputs[:, 2, :])
      # print(outputs)
      # Add the loss and batch size to the list:
      loss = outputs[0]
      losses.append(loss.item())
      nums.append(len(xb))

      loss.backward()

      optimizer.step()
      model.zero_grad()

    # Compute the average cost over one epoch:
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)

    # Now do the same thing for validation:

    model.eval()

    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        # inputs = xb.to(device)
        outputs = model(inputs[:, 0, :], token_type_ids=inputs[:, 1, :], labels=inputs[:, 2, :])
        losses.append(outputs[0].item())
        nums.append(len(xb))


    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)

    print('\n--- Epoch #{} finished --- Training cost: {} / Validation cost: {}'.format(i, train_cost, val_cost))

In [10]:
# Move the model to the GPU:
device = torch.device('cuda')
model.to(device)

# Fine-tune GPT2 for two epochs:
optimizer = AdamW(model.parameters())
optimizer.step()
fit(model, optimizer, train_loader, val_loader, epochs=3)


--- Starting epoch #0 ---


Validation: 100%|██████████| 2/2 [00:00<00:00, 19.59it/s]



--- Epoch #0 finished --- Training cost: 1.431231556772904 / Validation cost: 0.8193696141242981

--- Starting epoch #1 ---


Validation: 100%|██████████| 2/2 [00:00<00:00, 18.94it/s]



--- Epoch #1 finished --- Training cost: 1.025867523594669 / Validation cost: 0.44665107131004333

--- Starting epoch #2 ---


Validation: 100%|██████████| 2/2 [00:00<00:00, 20.39it/s]


--- Epoch #2 finished --- Training cost: 0.7234174454184007 / Validation cost: 0.3510701656341553


In [11]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
  """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
      Args:
          logits: logits distribution shape (batch size x vocabulary size)
          top_k > 0: keep only top k tokens with highest probability (top-k filtering).
          top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
              Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
      From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  top_k = min(top_k, logits.size(-1))  # Safety check
  if top_k > 0:
    # Remove all tokens with a probability less than the last token of the top-k
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = filter_value

  if top_p > 0.0:
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    # Remove tokens with cumulative probability above the threshold
    sorted_indices_to_remove = cumulative_probs > top_p
    # Shift the indices to the right to keep also the first token above the threshold
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    # scatter sorted tensors to original indexing
    indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
    logits[indices_to_remove] = filter_value
  return logits

In [12]:

# From HuggingFace, adapted to work with the context/character separation:
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=0, top_p=0.0,
                    repetition_penalty=1.0,
                    device='cpu'):
  context = torch.tensor(context, dtype=torch.long, device=device)
  context = context.unsqueeze(0).repeat(num_samples, 1)
  generated = context

  with torch.no_grad():
    for _ in trange(length):

      inputs = {'input_ids': generated}
      if segments_tokens != None:
        inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples,
                                                                                                          1)

      outputs = model(
        **inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet/CTRL (cached hidden-states)
      next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

      # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
      for i in range(num_samples):
        for _ in set(generated[i].tolist()):
          next_token_logits[i, _] /= repetition_penalty

      filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
      if temperature == 0:  # greedy sampling:
        next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
      else:
        next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
      generated = torch.cat((generated, next_token), dim=1)
  return generated

In [13]:
torch.save(model.state_dict(), "IceKingModel")

In [11]:
model.load_state_dict(torch.load("IceKingModel"))

<All keys matched successfully>

In [14]:
context = "Who are you?"

context_tkn = tokenizer.additional_special_tokens_ids[0]
character_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [character_tkn] * 64
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [character_tkn]

# Move the model back to the CPU for inference:
model.to(torch.device('cpu'))

# Generate 20 samples of max length 20
generated = sample_sequence(
  model,
  length=20,
  context=input_ids,
  segments_tokens=segments,
  num_samples=20,
  temperature=1,
  top_k=0,
  top_p=0
)

for g in generated:
  character = tokenizer.decode(g.squeeze().tolist())
  character = character.split('<|endoftext|>')[0]
  character = character.split('<context>')[1]
  character = character.split('<character>')
  print(character[0], character[1])

NameError: name 'character_tkn' is not defined